# **<font color=white> 07.Regularized Model-Ridge Code 실습**

[목적]
  1. Linear Regression
    - 단순 Linear Regression을 활용하여 변수의 중요도 및 방향성을 알아봄
    - 매우 심플한 모델이기 때문에 사이즈가 큰 데이터에 적합하지 않음
    - 하지만 설명력에서는 큰 장점이 있음
  2. Ridge Regression
    - Regularized Linear Model을 활용하여 Overfitting을 방지함
    - Hyperparameter lamba를 튜닝할 때 for loop 뿐만 아니라 GridsearchCV를 통해 돌출해봄
  3. Regularized Linear Models의 경우 X's Scaling을 필수적으로 진행해야함

[Process]
  1. Define X's & Y
  2. Split Train & Valid dataset
  3. Modeling
  4. Model 해석

In [23]:
# Package
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# Sklearn Toy Data
from sklearn.datasets import load_diabetes

In [24]:
# Data Loading (당뇨병)
data = load_diabetes()

In [25]:
print('>>>>X Shape : {}'.format(data['data'].shape))
print('>>>>Y Shape : {}'.format(data['target'].shape))

>>>>X Shape : (442, 10)
>>>>Y Shape : (442,)


In [26]:
# 데이터 설명
print(data['DESCR'])

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

In [27]:
pd.DataFrame(data['data'], columns=data['feature_names']).describe().applymap(lambda x: f"{x:0.2f}")

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
count,442.00,442.00,442.00,442.00,442.00,442.00,442.00,442.00,442.00,442.00
mean,-0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00
std,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
min,-0.11,-0.04,-0.09,-0.11,-0.13,-0.12,-0.10,-0.08,-0.13,-0.14
25%,-0.04,-0.04,-0.03,-0.04,-0.03,-0.03,-0.04,-0.04,-0.03,-0.03
50%,0.01,-0.04,-0.01,-0.01,-0.00,-0.00,-0.01,-0.00,-0.00,-0.00
75%,0.04,0.05,0.03,0.04,0.03,0.03,0.03,0.03,0.03,0.03
max,0.11,0.05,0.17,0.13,0.15,0.20,0.18,0.19,0.13,0.14


In [28]:
data = pd.read_csv('https://www4.stat.ncsu.edu/~boos/var.select/diabetes.tab.txt', sep='\t')

In [29]:
data.describe()

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
count,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000
mean,48.518100,1.468326,26.375792,94.647014,189.140271,115.439140,49.788462,4.070249,4.641411,91.260181,152.133484
std,13.109028,0.499561,4.418122,13.831283,34.608052,30.413081,12.934202,1.290450,0.522391,11.496335,77.093005
min,19.000000,1.000000,18.000000,62.000000,97.000000,41.600000,22.000000,2.000000,3.258100,58.000000,25.000000
25%,38.250000,1.000000,23.200000,84.000000,164.250000,96.050000,40.250000,3.000000,4.276700,83.250000,87.000000
50%,50.000000,1.000000,25.700000,93.000000,186.000000,113.000000,48.000000,4.000000,4.620050,91.000000,140.500000
75%,59.000000,2.000000,29.275000,105.000000,209.750000,134.500000,57.750000,5.000000,4.997200,98.000000,211.500000
max,79.000000,2.000000,42.200000,133.000000,301.000000,242.400000,99.000000,9.090000,6.107000,124.000000,346.000000


In [92]:
# X's & Y Split
Y = data['Y']
X = data.drop(columns=['Y']) 
X = pd.get_dummies(X, columns=['SEX'])

In [31]:
X

,AGE,BMI,BP,S1,S2,S3,S4,S5,S6,SEX_1,SEX_2
0,59,32.1,101.00,157,93.2,38.0,4.00,4.8598,87,False,True
1,48,21.6,87.00,183,103.2,70.0,3.00,3.8918,69,True,False
2,72,30.5,93.00,156,93.6,41.0,4.00,4.6728,85,False,True
3,24,25.3,84.00,198,131.4,40.0,5.00,4.8903,89,True,False
4,50,23.0,101.00,192,125.4,52.0,4.00,4.2905,80,True,False
...,...,...,...,...,...,...,...,...,...,...,...
437,60,28.2,112.00,185,113.8,42.0,4.00,4.9836,93,False,True
438,47,24.9,75.00,225,166.0,42.0,5.00,4.4427,102,False,True
439,60,24.9,99.67,162,106.6,43.0,3.77,4.1271,95,False,True
440,36,30.0,95.00,201,125.2,42.0,4.79,5.1299,85,True,False


[Data Split]

- Data Split을 진행할 때 BigData의 경우 꼭 indexing을 추출하여 모델에 적용시켜야 함
- 이유는 Data Split하여 새로운 Data set을 만들 경우 메모리에 부담을 주기 때문

In [32]:
X.shape[0]

442

In [33]:
idx = list(range(X.shape[0]))
train_idx, valid_idx = train_test_split(idx, test_size=0.3, random_state=2023)
print(">>>> # of Train data : {}".format(len(train_idx)))
print(">>>> # of valid data : {}".format(len(valid_idx)))

>>>> # of Train data : 309
>>>> # of valid data : 133


In [34]:
# Linear Regression
results = LinearRegression().fit(X.iloc[train_idx], Y.iloc[train_idx])

In [35]:
import scipy
from sklearn import metrics

def sse(clf, X, y):
    """Calculate the standard squared error of the model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    float
        The standard squared error of the model.
    """
    y_hat = clf.predict(X)
    sse = np.sum((y_hat - y) ** 2)
    return sse / X.shape[0]


def adj_r2_score(clf, X, y):
    """Calculate the adjusted :math:`R^2` of the model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    float
        The adjusted :math:`R^2` of the model.
    """
    n = X.shape[0]  # Number of observations
    p = X.shape[1]  # Number of features
    r_squared = metrics.r2_score(y, clf.predict(X))
    return 1 - (1 - r_squared) * ((n - 1) / (n - p - 1))


def coef_se(clf, X, y):
    """Calculate standard error for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of standard errors for the beta coefficients.
    """
    n = X.shape[0]
    X1 = np.hstack((np.ones((n, 1)), np.matrix(X)))
    se_matrix = scipy.linalg.sqrtm(
        metrics.mean_squared_error(y, clf.predict(X)) *
        np.linalg.inv(X1.T * X1)
    )
    return np.diagonal(se_matrix)


def coef_tval(clf, X, y):
    """Calculate t-statistic for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of t-statistic values.
    """
    a = np.array(clf.intercept_ / coef_se(clf, X, y)[0])
    b = np.array(clf.coef_ / coef_se(clf, X, y)[1:])
    return np.append(a, b)


def coef_pval(clf, X, y):
    """Calculate p-values for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of p-values.
    """
    n = X.shape[0]
    t = coef_tval(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p

def summary(clf, X, y, xlabels=None):
    """
    Output summary statistics for a fitted regression model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    xlabels : list, tuple
        The labels for the predictors.
    """
    # Check and/or make xlabels
    ncols = X.shape[1]
    if xlabels is None:
        xlabels = np.array(
            ['x{0}'.format(i) for i in range(1, ncols + 1)], dtype='str')
    elif isinstance(xlabels, (tuple, list)):
        xlabels = np.array(xlabels, dtype='str')
    # Make sure dims of xlabels matches dims of X
    if xlabels.shape[0] != ncols:
        raise AssertionError(
            "Dimension of xlabels {0} does not match "
            "X {1}.".format(xlabels.shape, X.shape))
    # Create data frame of coefficient estimates and associated stats
    coef_df = pd.DataFrame(
        index=['_intercept'] + list(xlabels),
        columns=['Estimate', 'Std. Error', 't value', 'p value']
    )
    try:
        coef_df['Estimate'] = np.concatenate(
            (np.round(np.array([clf.intercept_]), 6), np.round((clf.coef_), 6)))
    except Exception as e:
        coef_df['Estimate'] = np.concatenate(
            (
                np.round(np.array([clf.intercept_]), 6),
                np.round((clf.coef_), 6)
            ), axis = 1
    )[0,:]
    coef_df['Std. Error'] = np.round(coef_se(clf, X, y), 6)
    coef_df['t value'] = np.round(coef_tval(clf, X, y), 4)
    coef_df['p value'] = np.round(coef_pval(clf, X, y), 6)
    # Output results
    print('Coefficients:')
    print(coef_df.to_string(index=True))
    print('---')
    print('R-squared:  {0:.6f},    Adjusted R-squared:  {1:.6f},    MSE: {2:.1f}'.format(
        metrics.r2_score(y, clf.predict(X)), adj_r2_score(clf, X, y), sse(clf, X, y)))

In [51]:
X.head()

,AGE,BMI,BP,S1,S2,S3,S4,S5,S6,SEX_1,SEX_2
0,59.0,32.1,101.0,157.0,93.2,38.0,4.0,4.8598,87.0,False,True
1,48.0,21.6,87.0,183.0,103.2,70.0,3.0,3.8918,69.0,True,False
2,72.0,30.5,93.0,156.0,93.6,41.0,4.0,4.6728,85.0,False,True
3,24.0,25.3,84.0,198.0,131.4,40.0,5.0,4.8903,89.0,True,False
4,50.0,23.0,101.0,192.0,125.4,52.0,4.0,4.2905,80.0,True,False


In [95]:
X.loc[X.SEX_1 == True, 'SEX_1'] = float(1)
X.loc[X.SEX_1 == False, 'SEX_1'] = float(0)
X.loc[X.SEX_2 == True, 'SEX_2'] = float(1)
X.loc[X.SEX_2 == False, 'SEX_2'] = float(0)

In [98]:
X['AGE'] = X['AGE'].astype('float')
X['S1'] = X['S1'].astype('float')
X['S6'] = X['S6'].astype('float')
X['SEX_1'] = X['SEX_1'].astype('float')
X['SEX_2'] = X['SEX_2'].astype('float')

In [102]:
summary(results, X.iloc[valid_idx], Y.iloc[valid_idx], xlabels=X.columns) 

Coefficients:
              Estimate                  Std. Error         t value   p value
_intercept -353.422717  6.316113e+01+2.689859e+08j -0.0000+0.0000j  0.999999
AGE          -0.241046  9.945130e-01+0.000000e+00j -0.2424+0.0000j  0.808865
BMI           5.364734  1.330692e+00+0.000000e+00j  4.0315-0.0000j  0.000093
BP            0.973515  3.984600e-01+0.000000e+00j  2.4432-0.0000j  0.015879
S1           -1.128987  3.358510e-01+0.000000e+00j -3.3616+0.0000j  0.001014
S2            0.935342  3.569610e-01-0.000000e+00j  2.6203+0.0000j  0.009815
S3            0.295834  5.087240e-01-0.000000e+00j  0.5815+0.0000j  0.561881
S4            2.577375  1.011274e+01+0.000000e+00j  0.2549-0.0000j  0.799225
S5           72.840272  2.161884e+01+0.000000e+00j  3.3693-0.0000j  0.000988
S6            0.292290  4.446440e-01+0.000000e+00j  0.6574-0.0000j  0.512095
SEX_1        10.444984  9.014236e+00+2.689859e+08j  0.0000-0.0000j  1.000000
SEX_2       -10.444984  2.979639e+01+2.689859e+08j -0.0000+0.0

In [104]:
# Scaling
scaler = MinMaxScaler().fit(X.iloc[train_idx])
X_scal = scaler.transform(X)
X_scal = pd.DataFrame(X_scal, columns=X.columns)

In [103]:
X_scal

,AGE,BMI,BP,S1,S2,S3,S4,S5,S6,SEX_1,SEX_2
0,0.666667,0.573840,0.565217,0.294118,0.297578,0.197368,0.318471,0.562217,0.439394,0.0,1.0
1,0.483333,0.130802,0.362319,0.421569,0.355248,0.618421,0.159236,0.222437,0.166667,1.0,0.0
2,0.883333,0.506329,0.449275,0.289216,0.299885,0.236842,0.318471,0.496578,0.409091,0.0,1.0
3,0.083333,0.286920,0.318841,0.495098,0.517878,0.223684,0.477707,0.572923,0.469697,1.0,0.0
4,0.516667,0.189873,0.565217,0.465686,0.483276,0.381579,0.318471,0.362385,0.333333,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
437,0.683333,0.409283,0.724638,0.431373,0.416378,0.250000,0.318471,0.605672,0.530303,0.0,1.0
438,0.466667,0.270042,0.188406,0.627451,0.717416,0.250000,0.477707,0.415810,0.666667,0.0,1.0
439,0.683333,0.270042,0.545942,0.318627,0.374856,0.263158,0.281847,0.305030,0.560606,0.0,1.0
440,0.283333,0.485232,0.478261,0.509804,0.482122,0.250000,0.444268,0.657026,0.409091,1.0,0.0


In [105]:
# Linear Regression
results = LinearRegression().fit(X_scal.iloc[train_idx], Y.iloc[train_idx])
summary(results, X_scal.iloc[valid_idx], Y.iloc[valid_idx], xlabels=X_scal.columns)

Coefficients:
              Estimate                  Std. Error         t value   p value
_intercept   -2.765884  4.245773e+08-0.000000e+00j -0.0000-0.0000j  1.000000
AGE         -14.462769  2.388601e+01-0.000000e+00j -0.6055-0.0000j  0.545892
BMI         127.144195  3.182505e+01-0.000000e+00j  3.9951+0.0000j  0.000107
BP           67.172560  2.824374e+01-0.000000e+00j  2.3783+0.0000j  0.018825
S1         -230.313267  1.638999e+02+0.000000e+00j -1.4052+0.0000j  0.162309
S2          162.188278  1.158883e+02+0.000000e+00j  1.3995-0.0000j  0.164002
S3           22.483360  7.137033e+01+0.000000e+00j  0.3150-0.0000j  0.753241
S4           16.185916  5.675401e+01-0.000000e+00j  0.2852+0.0000j  0.775942
S5          207.514650  5.071283e+01+0.000000e+00j  4.0920-0.0000j  0.000074
S6           19.291168  3.428908e+01-0.000000e+00j  0.5626+0.0000j  0.574659
SEX_1        10.444984  4.245773e+08+0.000000e+00j  0.0000-0.0000j  1.000000
SEX_2       -10.444984  4.245773e+08-0.000000e+00j -0.0000-0.0

[Ridge Regression]
 - Hyperparameter Tuning using for Loop
 - Hyperparameter Tuning using GridSearchCV

[Ridge Regression Parameters]
   - Package : https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html
   - alpha : L2-norm Penalty Term 
     - alpha : 0 일 때, Just Linear Regression
   - fit_intercept : Centering to zero
     - 베타0를 0로 보내는 것 (베타0는 상수이기 때문에)
   - max_iter : Maximum number of interation
     - Loss Function의 Ridge Penalty Term은 Closed Form 값이기는 하지만 값을 찾아 나감
     - Penalty Term : (1 / (2 * n_samples)) * ||y - Xw||^2_2 + alpha * ||w||_2

In [106]:
penelty = [0.00001, 0.00005, 0.0001, 0.001, 0.01, 0.1, 0.3, 0.5, 0.6, 0.7, 0.9, 1, 10]

# Using For Loop !! 
# Ridge Regression
# select alpha by checking R2, MSE, RMSE
for a in penelty:
    model = Ridge(alpha=a).fit(X_scal.iloc[train_idx], Y.iloc[train_idx])
    score = model.score(X_scal.iloc[valid_idx], Y.iloc[valid_idx])
    pred_y = model.predict(X_scal.iloc[valid_idx])
    mse = mean_squared_error(Y.iloc[valid_idx], pred_y)
    print("Alpha:{0:.5f}, R2:{1:.7f}, MSE:{2:.7f}, RMSE:{3:.7f}".format(a, score, mse, np.sqrt(mse))) 

Alpha:0.00001, R2:0.5301655, MSE:3084.6095744, RMSE:55.5392616
Alpha:0.00005, R2:0.5301672, MSE:3084.5988322, RMSE:55.5391648
Alpha:0.00010, R2:0.5301692, MSE:3084.5854446, RMSE:55.5390443
Alpha:0.00100, R2:0.5302048, MSE:3084.3519225, RMSE:55.5369420
Alpha:0.01000, R2:0.5304637, MSE:3082.6522133, RMSE:55.5216373
Alpha:0.10000, R2:0.5304511, MSE:3082.7343410, RMSE:55.5223769
Alpha:0.30000, R2:0.5294946, MSE:3089.0142040, RMSE:55.5789007
Alpha:0.50000, R2:0.5285641, MSE:3095.1232917, RMSE:55.6338323
Alpha:0.60000, R2:0.5280578, MSE:3098.4477026, RMSE:55.6637018
Alpha:0.70000, R2:0.5275205, MSE:3101.9751863, RMSE:55.6953785
Alpha:0.90000, R2:0.5263592, MSE:3109.5989675, RMSE:55.7637783
Alpha:1.00000, R2:0.5257398, MSE:3113.6657269, RMSE:55.8002305
Alpha:10.00000, R2:0.4513724, MSE:3601.9109235, RMSE:60.0159222


In [107]:
model_best = Ridge(alpha=0.01).fit(X_scal.iloc[train_idx], Y.iloc[train_idx])
summary(model_best, X_scal.iloc[valid_idx], Y.iloc[valid_idx], xlabels = X_scal.columns)

Coefficients:
              Estimate                  Std. Error         t value   p value
_intercept    3.351510  4.244424e+08-0.000000e+00j  0.0000+0.0000j  1.000000
AGE         -14.279970  2.364516e+01+0.000000e+00j -0.6039+0.0000j  0.546928
BMI         127.331685  3.179837e+01+0.000000e+00j  4.0043-0.0000j  0.000103
BP           67.279481  2.817913e+01-0.000000e+00j  2.3876+0.0000j  0.018378
S1         -203.277222  1.638013e+02+0.000000e+00j -1.2410+0.0000j  0.216807
S2          141.209566  1.158346e+02+0.000000e+00j  1.2191-0.0000j  0.224995
S3           10.619292  7.128066e+01-0.000000e+00j  0.1490+0.0000j  0.881798
S4           13.868910  5.674602e+01+0.000000e+00j  0.2444-0.0000j  0.807298
S5          198.003880  5.059235e+01+0.000000e+00j  3.9137-0.0000j  0.000145
S6           19.251667  3.422230e+01+0.000000e+00j  0.5625-0.0000j  0.574697
SEX_1        10.473795  4.244423e+08-0.000000e+00j  0.0000+0.0000j  1.000000
SEX_2       -10.473795  4.244423e+08-0.000000e+00j -0.0000-0.0

In [108]:
# Using GridSearchCV
ridge_cv=RidgeCV(alphas=penelty, cv=5)
model = ridge_cv.fit(X_scal.iloc[train_idx], Y.iloc[train_idx])
print("Best Alpha:{0:.5f}, R2:{1:.4f}".format(model.alpha_, model.best_score_))

Best Alpha:0.90000, R2:0.4419


In [109]:
# GridSearchCV Result
model_best = Ridge(alpha=model.alpha_).fit(X_scal.iloc[train_idx], Y.iloc[train_idx])
score = model_best.score(X_scal.iloc[valid_idx], Y.iloc[valid_idx])
pred_y = model_best.predict(X_scal.iloc[valid_idx])
mse = np.sqrt(mean_squared_error(Y.iloc[valid_idx], pred_y))
print("Alpha:{0:.5f}, R2:{1:.7f}, MSE:{2:.7f}, RMSE:{3:.7f}".format(0.01, score, mse, np.sqrt(mse)))
summary(model_best, X_scal.iloc[valid_idx], Y.iloc[valid_idx], xlabels=X_scal.columns)

Alpha:0.01000, R2:0.5263592, MSE:55.7637783, RMSE:7.4675149
Coefficients:
              Estimate                  Std. Error         t value   p value
_intercept   42.643650  4.262934e+08+0.000000e+00j  0.0000-0.0000j  1.000000
AGE         -10.492753  2.362621e+01+0.000000e+00j -0.4441+0.0000j  0.657687
BMI         121.677019  3.188409e+01+0.000000e+00j  3.8162-0.0000j  0.000207
BP           66.649695  2.820463e+01+0.000000e+00j  2.3631-0.0000j  0.019584
S1          -21.091765  1.644287e+02-0.000000e+00j -0.1283-0.0000j  0.898128
S2           -5.450923  1.162868e+02+0.000000e+00j -0.0469+0.0000j  0.962684
S3          -55.208206  7.124774e+01+0.000000e+00j -0.7749+0.0000j  0.439798
S4           17.858217  5.701644e+01+0.000000e+00j  0.3132-0.0000j  0.754614
S5          119.080753  5.067553e+01+0.000000e+00j  2.3499-0.0000j  0.020263
S6           22.603065  3.422059e+01+0.000000e+00j  0.6605-0.0000j  0.510077
SEX_1        10.489062  4.262934e+08+0.000000e+00j  0.0000-0.0000j  1.000000
SE